###  Import Packages

In [ ]:
from datetime import datetime
import pandas as pd 
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.express as px
from scipy.stats import lognorm
from statsmodels.distributions.empirical_distribution import ECDF


### Load Binance API Kbar

In [2]:

btc_kbar = pd.read_csv('btc.csv')
eth_kbar = pd.read_csv('eth.csv')
sol_kbar = pd.read_csv('sol.csv')
xrp_kbar = pd.read_csv('xrp.csv')



In [3]:
def convert_to_dateime(df) : 
    df = df[['open_time', 'open', 'high', 'low', 'close', 'volume', 'taker_buy_volume', 'taker_sell_volume']]
    df['open_time'] = pd.to_datetime(df['open_time'])
    return df 


btc_kbar = convert_to_dateime(btc_kbar)
eth_kbar = convert_to_dateime(eth_kbar)
sol_kbar = convert_to_dateime(sol_kbar)
xrp_kbar = convert_to_dateime(xrp_kbar)


/tmp/ipykernel_245622/1814961824.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['open_time'] = pd.to_datetime(df['open_time'])
/tmp/ipykernel_245622/1814961824.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['open_time'] = pd.to_datetime(df['open_time'])
/tmp/ipykernel_245622/1814961824.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

### Rolling Volume Bars Function

In [4]:
# 每日平均交易量
def rolling_avg_daily_volume(df, col):
    vol = df.groupby([df[col].dt.date])['volume'].sum().reset_index()
    return vol


In [5]:
# Single Volume Bar Function
# frequency : 每根 Volume Bar 交易量大小
def generate_single_volumebar(trades, frequency=10):
    
    times = trades['open_time']
    volumes = trades['volume']
    buy_volume = trades['taker_buy_volume']
    sell_volume = trades['taker_sell_volume']


    open_ = trades['open']
    high_ = trades['high']
    low_ = trades['low']
    close_ = trades['close']

 
    candle_counter = 0 
    vol = 0
    lasti = 0

    result = {}

    for i in range(len(times)):
        vol += volumes[i]
        if vol >= frequency:

            volumes_result = np.sum(volumes[lasti:i+1])  # volume
            buy_volumes_result = np.sum(buy_volume[lasti:i+1])  # volume
            sell_volumes_result = np.sum(sell_volume[lasti:i+1])  # volume


            open_result = open_[lasti] # open price
            high_result = np.max(high_[lasti:i+1]) # high price
            low_result = np.min(low_[lasti:i+1]) # low price
            close_result = close_[i] # close price

            result[times[i]] = [volumes_result, buy_volumes_result, sell_volumes_result, open_result, high_result, low_result, close_result, frequency]                      
            lasti = i+1
            vol = 0
            break
    
    
    #print(result)
    result = pd.DataFrame(result).T.reset_index()
    result.columns = ['open_time', 'volume', 'buy_volume', 'sell_volume', 'open', 'high', 'low', 'close', 'V']

    return result


In [6]:
## Rolling Volume Bars

def get_rolling_volumebars(df_kbar) : 

    # 使用過去 1 年日均交易量
    freq_end_time = df_kbar['open_time'].min() + pd.DateOffset(years = 1)
    df_rolling_volumebar = pd.DataFrame()

    # V = 日均交易量/daily_vol_devisor_array
    daily_vol_devisor_array = 12

    while freq_end_time < df_kbar['open_time'].max() : 

        try : 

            # 計算 V 範圍 : 某根 Volume Bar 前一年
            freq_start_time = freq_end_time - pd.DateOffset(years = 1)

            df_kbar_freq = df_kbar[(df_kbar['open_time'] >= freq_start_time) & (df_kbar['open_time'] <  freq_end_time)]

            # 計算該 Volume Bar 過去一年的日平均交易量
            open_time_groupby = df_kbar_freq['open_time'] - pd.DateOffset(hours = df_kbar_freq['open_time'].min().hour) - pd.DateOffset(minutes = df_kbar_freq['open_time'].min().minute)
            df_kbar_freq = df_kbar_freq.assign(open_time_groupby = open_time_groupby)

            df_avg_volume = rolling_avg_daily_volume(df_kbar_freq, col = 'open_time_groupby')
            avg_volume = df_avg_volume['volume'].mean()/daily_vol_devisor_array

            # 計算下一根 Volume Bar 
            df_kbar_filter  = df_kbar[df_kbar['open_time'] >= freq_end_time]
            df_kbar_filter = df_kbar_filter.reset_index(drop = True)
            
            volumebar = generate_single_volumebar(df_kbar_filter, frequency=avg_volume)
            new_freq_end_time = volumebar['open_time'][0]
            
            # 若同一分鐘內即符合 Volume Bar 大小，平移到下一分鐘計算。
            if new_freq_end_time == freq_end_time : 
                new_freq_end_time = new_freq_end_time + pd.DateOffset(minutes = 1)
            
            freq_end_time = new_freq_end_time
            print(freq_end_time)

            df_rolling_volumebar = pd.concat([df_rolling_volumebar, volumebar])


        
        except : 
            break

    return df_rolling_volumebar


In [7]:
df_rolling_volumebar_btc  = get_rolling_volumebars(btc_kbar)
df_rolling_volumebar_btc = df_rolling_volumebar_btc.reset_index(drop = True)
#df_rolling_volumebar_btc.to_pickle("df_rolling_volumebar_btc_12.pkl")


2018-08-17 04:32:00
2018-08-17 05:33:00
2018-08-17 06:28:00
2018-08-17 07:00:00
2018-08-17 07:44:00
2018-08-17 08:38:00
2018-08-17 09:31:00
2018-08-17 10:17:00
